In [1]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import unicodedata
import os

/home/sonic/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## read the movie conversation data

## Download the data from here - https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html  or

## run pull.sh in data folder to fetch the training and testing data

In [2]:
! data/pull.sh


--2018-03-08 00:32:57--  https://www.dropbox.com/s/ncfa5t950gvtaeb/test.enc?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.1, 2620:100:601c:1::a27d:601
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/content_link/FjAUszSXOEj1H0pWzR5jh44IliaQkdoNff37xn0FBpetcMjXsdXoyhrocWmwxC8X/file [following]
--2018-03-08 00:32:57--  https://dl.dropboxusercontent.com/content_link/FjAUszSXOEj1H0pWzR5jh44IliaQkdoNff37xn0FBpetcMjXsdXoyhrocWmwxC8X/file
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.6.6, 2620:100:601c:6::a27d:606
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.6.6|:443... connected.
HTTP request sent, awaiting response... 200 OK

    The file is already fully retrieved; nothing to do.

--2018-03-08 00:32:58--  https://www.dropbox.com/s/48ro4759jaikque/test.dec?dl=0
Resolving www.dropbox.com (www.d

In [3]:
path='data/'
print (os.listdir(path))

['train.dec', 'train.enc', 'test.dec', 'conversation.pickle', 'pull.sh', 'test.enc']


In [4]:
train_enc=[]
train_dec=[]
f1=open('data/train.enc','rb')
f2=open('data/train.dec','rb')
for i in f1.readlines():
    try:
        temp1=i.decode("utf-8",errors='replace')
#         print (temp1[0])
    except Exception as e:
        print ('***********')
        print (i)
        break
    train_enc.append(temp1)
    
for j in f2.readlines():
    try:
        temp2=j.decode("utf-8",errors='replace')
#         print (temp2[0])
    except Exception as e:
        print ('***********')
        print (i)
        break
    train_dec.append(temp2)

In [5]:
test_enc=[]
test_dec=[]
f1=open('data/test.enc','rb')
f2=open('data/test.dec','rb')
for i in f1.readlines():
    try:
        temp1=i.decode("utf-8",errors='replace')
#         print (temp1[0])
    except Exception as e:
        print ('***********')
        print (i)
        break
    test_enc.append(temp1)
    
for j in f2.readlines():
    try:
        temp2=j.decode("utf-8",errors='replace')
#         print (temp2[0])
    except Exception as e:
        print ('***********')
        print (i)
        break
    test_dec.append(temp2)

In [6]:
print (train_enc[0:2],train_dec[0:2])

['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\n', "You're asking me out.  That's so cute. What's your name again?\n"] ["Well, I thought we'd start with pronunciation, if that's okay with you.\n", 'Forget it.\n']


In [7]:
train_list=[]
for i,j in zip(train_enc,train_dec):
    train_list.append([i,j])

In [9]:
data=pd.DataFrame(train_list,columns=['person1','person2'])
data = data[:10000]
data.tail()

,person1,person2
9995,What the hell is that?\n,That's the Core: the gravity drive. The heart ...
9996,The safety circuit's failed!\n,We're losing atmosphere...\n
9997,I didn't see anything and I don't have to see ...,"Thank you for that scientific analysis, Mister..."
9998,"I can't believe this, I haven't gotten more th...",Smith's right. Neptune? There's nothing out th...
9999,30 hours to Neptune orbit.\n,"All boards are green, everything's five by fiv..."


## For doing preprocessing of text you can alter below code to work on both person1 and person2 columns

In [10]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')
import re
import numpy as np
# #stop=['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}','/','-']
# def number_removal(row):
#     data1 = row['Summary']
#     #print data1
#     #tokens = nltk.wordpunct_tokenize(data)
#     #print (type(data1))
#     if type(data1) not in [int,float]:
#         print type(data1)
#         line = re.sub(r"[^A-Za-z\s]", " ", data1.strip())
#         tokens = line.split()
#     else:
#         tokens=[]
#     return ' '.join(tokens)


# frequency_words_wo_stop = {}
# def generate_word_frequency(row):
#     data1 = row['Summary']
#     tokens = nltk.wordpunct_tokenize(data1)
#     token_list = []
#     for token in tokens:
# #         if token.lower() not in stop:
# #             token_list.append(token.lower())
#         if token.lower() in frequency_words_wo_stop:
#             count = frequency_words_wo_stop[token.lower()]
#             count = count + 1
#             frequency_words_wo_stop[token.lower()] = count
#         else:
#             frequency_words_wo_stop[token.lower()] = 1
    
#     return ','.join(token_list)

# from nltk.stem import WordNetLemmatizer
# wordnet_lemmatizer = WordNetLemmatizer()


# def receieve(query):
#     #data=pd.DataFrame([query])
#     #data.columns=['Summary']
#     data1=query
#     #print (data1.head())
#     data1['Summary'] = data1.apply(number_removal,axis=1)
    
#     data1['tokens'] = data1.apply(generate_word_frequency,axis=1)
#     print (data1.head())
#     big=[]
#     for i in data1['tokens']:
#         st=''
#         ls=[]
#         for j in i.split(','):
#             #print j
#             print (wordnet_lemmatizer.lemmatize(j))
#             ls.append(wordnet_lemmatizer.lemmatize(j))
#         #print ls
#         big.append(' '.join(ls))
#     data1['Summary_lem']=big
#     return data1['Summary_lem']
# data['Summary']=receieve(data[['Summary']])

In [11]:
# from nltk.stem import WordNetLemmatizer
# wordnet_lemmatizer = WordNetLemmatizer()
# def fun(dat):
#     big=[]
#     for i in dat['Question']:
#         st=''
#         ls=[]
#         for j in i.split():
#             #print j
#             ls.append(wordnet_lemmatizer.lemmatize(re.sub(r"[^A-Za-z\s]", " ", j.strip().lower())))
#         #print ls
#         big.append(' '.join(ls))
#     return big


# data['Question']=fun(data)

In [12]:
X=data['person1'].values
y=data['person2'].values

# skf = StratifiedKFold(y, n_folds=4)
# for train_index, test_index in skf:
#     #print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]

In [13]:
import json
import nltk
import gensim
import numpy as np
from gensim import corpora, models, similarities
import pickle

## Convert words into vectors using Google's word vectors
### Download from https://code.google.com/archive/p/word2vec/

In [14]:
! pwd
# model = gensim.models.Word2Vec.load('../data/5', binary='binary')
model = gensim.models.KeyedVectors.load_word2vec_format('../data/GoogleNews-vectors-negative300.bin', binary=True)   

/home/sonic/sonic/ourDirection/momus/NLG-LSTM-Keras


In [15]:
tok_x=[]
tok_y=[]
for i in range(len(X)):
    tok_x.append(nltk.word_tokenize(X[i].lower()))
    tok_y.append(nltk.word_tokenize(y[i].lower()))


In [16]:
tok_y[:2]

[['well',
  ',',
  'i',
  'thought',
  'we',
  "'d",
  'start',
  'with',
  'pronunciation',
  ',',
  'if',
  'that',
  "'s",
  'okay',
  'with',
  'you',
  '.'],
 ['forget', 'it', '.']]

In [17]:
sentend=np.ones((300,),dtype=np.float32) 

vec_x=[]
for sent in tok_x:
    sentvec = [model[w] for w in sent if w in model.vocab]
    vec_x.append(sentvec)
    
vec_y=[]
for sent in tok_y:
    sentvec = [model[w] for w in sent if w in model.vocab]
    vec_y.append(sentvec)           


In [18]:
for tok_sent in vec_x:
    tok_sent[14:]=[]
    tok_sent.append(sentend)

In [19]:
for tok_sent in vec_x:
    if len(tok_sent)<15:
        for i in range(15-len(tok_sent)):
            tok_sent.append(sentend) 

In [20]:
len(max(tok_y))

33

In [21]:
for tok_sent in vec_y:
    tok_sent[14:]=[]
    tok_sent.append(sentend)
    

for tok_sent in vec_y:
    if len(tok_sent)<15:
        for i in range(15-len(tok_sent)):
            tok_sent.append(sentend)

In [22]:
with open('data/conversation.pickle','wb') as f:
    pickle.dump([vec_x,vec_y],f)

In [23]:
# vec_x[0]

## Training

In [24]:
from keras.models import Sequential
from keras.layers.recurrent import LSTM,SimpleRNN
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


In [25]:
f=open('data/conversation.pickle','rb')
vec_x,vec_y=pickle.load(f)    
    
vec_x=np.array(vec_x,dtype=np.float32)
vec_y=np.array(vec_y,dtype=np.float32)    

In [26]:
x_train,x_test, y_train,y_test = train_test_split(vec_x, vec_y, test_size=0.2, random_state=1)

In [27]:
model=Sequential()
model.add(LSTM(units=300, input_shape=x_train.shape[1:],
               return_sequences=True, recurrent_initializer='glorot_normal', kernel_initializer='glorot_normal', activation='sigmoid'))
model.add(LSTM(units=300,input_shape=x_train.shape[1:],
               return_sequences=True, recurrent_initializer='glorot_normal', kernel_initializer='glorot_normal', activation='sigmoid'))
model.add(LSTM(units=300,input_shape=x_train.shape[1:],
               return_sequences=True, recurrent_initializer='glorot_normal', kernel_initializer='glorot_normal', activation='sigmoid'))
model.add(LSTM(units=300,input_shape=x_train.shape[1:],
               return_sequences=True, recurrent_initializer='glorot_normal', kernel_initializer='glorot_normal', activation='sigmoid'))
model.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 15, 300)           721200    
_________________________________________________________________
lstm_2 (LSTM)                (None, 15, 300)           721200    
_________________________________________________________________
lstm_3 (LSTM)                (None, 15, 300)           721200    
_________________________________________________________________
lstm_4 (LSTM)                (None, 15, 300)           721200    
Total params: 2,884,800
Trainable params: 2,884,800
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x_train, y_train, epochs=500, validation_data=(x_test, y_test))
model.save('data/LSTM500.h5')
# model.fit(x_train, y_train, nb_epoch=500,validation_data=(x_test, y_test))
# model.save('lstm_models/LSTM1000.h5')
# model.fit(x_train, y_train, nb_epoch=500,validation_data=(x_test, y_test))
# model.save('lstm_models/LSTM1500.h5')
# model.fit(x_train, y_train, nb_epoch=500,validation_data=(x_test, y_test))
# model.save('lstm_models/LSTM2000.h5')
# model.fit(x_train, y_train, nb_epoch=500,validation_data=(x_test, y_test))
# model.save('lstm_models/LSTM2500.h5')
# model.fit(x_train, y_train, nb_epoch=500,validation_data=(x_test, y_test))
# model.save('lstm_models/LSTM3000.h5')
# model.fit(x_train, y_train, nb_epoch=500,validation_data=(x_test, y_test))
# model.save('lstm_models/LSTM3500.h5')
# model.fit(x_train, y_train, nb_epoch=500,validation_data=(x_test, y_test))
# model.save('lstm_models/LSTM4000.h5')
# model.fit(x_train, y_train, nb_epoch=500,validation_data=(x_test, y_test))
# model.save('lstm_models/LSTM4500.h5')
# model.fit(x_train, y_train, nb_epoch=500,validation_data=(x_test, y_test))
# model.save('lstm_models/LSTM5000.h5')

Train on 8000 samples, validate on 2000 samples
Epoch 1/500
8000/8000 [==============================] - 123s 15ms/step - loss: -0.5895 - acc: 0.0266 - val_loss: -0.5953 - val_acc: 0.0271
Epoch 2/500
8000/8000 [==============================] - 124s 15ms/step - loss: -0.5898 - acc: 0.0272 - val_loss: -0.5957 - val_acc: 0.0235
Epoch 3/500
8000/8000 [==============================] - 123s 15ms/step - loss: -0.5901 - acc: 0.0271 - val_loss: -0.5956 - val_acc: 0.0247
Epoch 4/500
8000/8000 [==============================] - 124s 16ms/step - loss: -0.5904 - acc: 0.0281 - val_loss: -0.5965 - val_acc: 0.0263
Epoch 5/500
3040/8000 [==========>...................] - ETA: 1:15 - loss: -0.5884 - acc: 0.0288

In [31]:
x_train[1:].shape

(7999, 15, 300)

In [33]:
predictions=model.predict(x_test) 
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('../data/GoogleNews-vectors-negative300.bin', binary=True)   
[w2v_model.most_similar([predictions[10][i]])[0] for i in range(15)]

[('Laughing.', 0.5424548983573914),
 ('do', 0.5389987230300903),
 ('anyway', 0.5267603993415833),
 ('RUSH_Okay', 0.5002189874649048),
 ('Stupid_stupid', 0.46662095189094543),
 ('Stupid_stupid', 0.43449124693870544),
 ('reallllly', 0.3895082473754883),
 ('reallllly', 0.334981769323349),
 ('reallllly', 0.311807245016098),
 ('reallllly', 0.29463836550712585),
 ('reallllly', 0.27858787775039673),
 ('reallllly', 0.26468509435653687),
 ('Australian_sheepskin_boots', 0.2539001703262329),
 ('Australian_sheepskin_boots', 0.25108352303504944),
 ('Australian_sheepskin_boots', 0.24872171878814697)]

In [34]:
x='what this all about'

In [37]:
sentend=np.ones((300,),dtype=np.float32) 

sent=nltk.word_tokenize(x.lower())
sentvec = [w2v_model[w] for w in sent if w in w2v_model.vocab]

sentvec[14:]=[]
sentvec.append(sentend)
if len(sentvec)<15:
    for i in range(15-len(sentvec)):
        sentvec.append(sentend) 
sentvec=np.array([sentvec])

predictions = model.predict(sentvec)
#print (predictions)
outputlist=[w2v_model.most_similar([predictions[0][i]])[0][0] for i in range(15)]
output=' '.join(outputlist)
print (output)

Laughing. do anyway RUSH_Okay going Stupid_stupid reallllly reallllly reallllly reallllly reallllly reallllly Australian_sheepskin_boots Australian_sheepskin_boots Australian_sheepskin_boots


In [38]:
predictions[0][0],predictions[0][0].shape,predictions[0].shape

(array([ 0.15575278,  0.        ,  0.30011603,  0.70224357,  0.        ,
         0.06596337,  0.1735681 ,  0.        ,  0.30567577,  0.28602865,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.5       ,  0.4112463 ,  0.33293316,  0.18313767,  0.        ,
         0.        ,  0.26286513,  0.5       ,  0.        ,  0.13168032,
         0.15482874,  0.        ,  0.02708067,  0.10439767,  0.        ,
         0.        ,  0.36169937,  0.        ,  0.        ,  0.        ,
         0.07590561,  0.11171825,  0.18639541,  0.10530861,  0.5       ,
         0.46031433,  0.        ,  0.73098242,  0.        ,  0.10333289,
         0.11773524,  0.03792454,  0.        ,  0.03366207,  0.        ,
         0.        ,  0.361404  ,  0.06697976,  0.14651617,  0.02948635,
         0.22108823,  0.        ,  0.        ,  0.20671573,  0.        ,
         0.        ,  0.42755577,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.5       ,  0. 

In [39]:
model.predict_classes(sentvec)

array([[87, 87, 42, 42, 42, 42, 42, 87, 87, 87, 87, 42, 42, 42, 42]])

In [41]:
w2v_model.most_similar([predictions[0][0]])

[('Laughing.', 0.5428836345672607),
 ('anyway', 0.537278950214386),
 ('THE_PRESIDENT_Yes', 0.5365479588508606),
 ("O'REILLY_OK", 0.534572184085846),
 ('RUSH_Okay', 0.53199303150177),
 ('Stupid_stupid', 0.5310018658638),
 ('ÔI', 0.5272243618965149),
 ('Golly_gee', 0.5240897536277771),
 ('Mr._DIONNE', 0.5185691118240356),
 ('RUSH_Yeah', 0.5185155868530273)]

## Feel free to improve it and provide feedback